## Programming Assignment 2 - Part 2
### Cpts 355 - Fall 2015
### An Interpreter for a Postscript-like Language

## Assigned Sept. 21, 2015

### Assigned Sept. 21, 2015
### Due Monday, Oct. 5, 2015
Continue developing your code in the `sps.ipynb` file. You will need to copy some cells from this notebook into your notebook. I strongly encourage you to save a copy of periodically so you can go back in time if you really mess something up. Using a version control system like git or mercurial would be a good idea!

When you are finished, upload `sps.ipynb` on the course Turnin Page for the Interpreter Assignment Part 2. (Don't 
upload using an upload page for any other assignment. Wait until the upload page for this assignment is ready.
If you think it should be ready and it isn't, [email me](mailto:hauser@eecs.wsu.edu).

The entire interpreter project (Parts 1 and Part 2 together) will count for 10% of your course grade. This second part is worth 80% of that 10%.

### This assignment is to be your own work. Refer to the course academic integrity statement in the syllabus.

## The problem
In this assignment you will write an interpreter in Python for a small PostScript-like language, concentrating on key computational features of the abstract machine, omitting all PS features related to graphics, and using a somewhat-simplified syntax.

The simplified language, SPS, has the following features of PS
* integer constants, e.g. `123`: in python3 there is no practical limit on the size of integers
* boolean constants, `true` and `false` (Note that the boolean constants in python are `True` and `False`)
* name constants, e.g. `/fact`: start with a `/` and letter followed by an arbitrary sequence of letters and numbers
* names to be looked up in the dictionary stack, e.g. `fact`: as for name constants, without the `/`
* code constants: code between matched curly braces `{` ... `}`
* built-in operators on numbers: `add`, `sub`, `mul`, `div`, `eq`, `lt`, `gt`
* built-in operators on boolean values: `and`, `or`, `not`; these take boolean operands only. Anything else is an error.
* built-in sequencing operators: `if`, `ifelse`; make sure that you understand the order of the operands on the stack. Play with ghostscript if necessary to help understand what is happening.
* stack operators: `dup`, `exch`, `pop`
* dictionary creation operator: `dictz`; takes no operands
* dictionary stack manipulation operators: `begin`, `end`. `begin` requires one dictionary operand on the operand stack; `end` has no operands.
* name definition operator: `def`. This requires two operands, a name and a value
* defining (using `def`) and calling functions
* stack printing operator (prints contents of stack without changing it): `stack`
* top-of-stack printing operator (pops the top element of the stack and prints it): `=`


## Requirements for Part 2 (Due Oct. 5)
In Part 2 you will continue building the interpreter, making use of everything you built in Part 1. The pieces needed to complete the interpreter are
* Parsing Postscript code
* Handling of *code arrays*
* Handling the `if` and `ifelse` operators
* Function calling
* Actually interpreting SPS programs

### Parsing 
Parsing is the process by which a program is converted to a data structure that can be further processed by an interpreter or compiler. Our SPS programs are very simple to parse. Essentially all we need to do is convert the continuous input text to a list of *tokens* and convert each token to our chosen representation for it. One way to think of tokens is as the minimum-sized "interesting" chunks of the text of a program. In SPS the tokens are: multidigit numbers with optional negative sign, `true` and `false`, multi-character names (with and without a preceding `/`), and the curly brace characters. We've already decided about how some of these will be represented: numbers as Python integers, names as Python strings, booleans as Python booleans, etc. It turns out that what we want for curly braces is not to represent the characters themselves, but rather to represent things falling between the braces, which we do using *code arrays*. Part of parsing is to identify and represent these code arrays, for which we will use Python lists.

### Uses of code arrays 
Recall that a code array is pushed on the stack as a single unit when it is read from the input. Once a code array is on the stack several things can happen: 
* if it is the `then` or `else` part of an `if` or `ifelse` it is recursively interpreted as part of the evaluation of the `if` or `ifelse`. (We will get to interpreting momentarily).
* if it is the top item on the stack when a `def` is executed, it is stored as the value of the name defined by the `def`.
Finally, if when a name is looked up you find that its value is a code array, the code array is recursively interpreted. 

### Key insight
A key insight is that a complete SPS program is essentially a code array. It doesn't have curly braces around it but it is a chunk of code that needs to be interpreted. This suggests how to proceed: convert the SPS program (a string of text) into an array of tokens and code arrays. Define a Python function `interpret` that takes one of these arrays as input and processes it. The `if` and `ifelse` operators choose zero or one of their operands to *recursively* `interpret`. When a name lookup produce a code array as its result, *recursively* `interpret` it,
thus implementing *Postscript function calls*.

### Parsing revisited
Parsing converts an SPS program in the form a string to a program in the form of a code array. It will work in two stages: first, convert all the string to a list of tokens. And then convert the token list to a code array. The difference between the two will be that in the code array, everything between matching curly braces will be represented as a single element (which itself is a code array). In the process of converting from token list to code array the curly braces will disappear, and the string representations of numbers and booleans will be
converted to Python ints and bools.

#### Tokenizing

In [6]:
# For tokenizing we'll use the re package for Regular Expressions. (copy this to your own sps.ipynb notebook) 
import re

def tokenize(s):
    return re.findall("/?[a-zA-Z][a-zA-Z0-9_]*|[-]?[0-9]+|[}{]+|%.*|[^ \t\n]", s)
   


In [7]:
# See what tokenize does
print (tokenize(
"""
/fact {
dictz
begin
/n exch def
n 2 lt
{1}
{n -1 add fact n mul }
ifelse
end
}def
5 fact =
"""))

['/fact', '{', 'dictz', 'begin', '/n', 'exch', 'def', 'n', '2', 'lt', '{', '1', '}', '{', 'n', '-1', 'add', 'fact', 'n', 'mul', '}', 'ifelse', 'end', '}', 'def', '5', 'fact', '=']


#### Grouping and converting to Python representations for numbers and booleans
The output of `tokenize` isn't fully suitable because things between matching curly braces are not themselves grouped into a code array. We need to convert the output for the above example to 
```
['/fact', ['dictz', 'begin', '/n', 'exch', 'def', 'n', 2, 'lt', [1], ['n', -1, 'add', 'fact', 'n', 'mul'], 'ifelse', 'end'], 'def', 5, 'fact', '=']
```
Notice how in addition to grouping tokens between curly braces into lists, we've also converted the strings that represent numbers to Python numbers; if there were any booleans, those should have been converted to Python booleans as well.

The main issue in how to convert to a code array is how to group things that fall in between matching 
curly braces. In class on Friday, Sept. 18, we went over a couple of strategies for this. I reproduce here the
code that I showed using a python *iterator*. 
```
# The it argument is an iterator that returns left or right parenthesis characters. 
# The sequence of characters returned by the iterator should represent a string of properly nested 
# parentheses pairs, from which the leading '(' has already been removed. If the 
# parenteses are not properly nested, returns False.
def groupMatching(it):
    res = ['(']
    for c in it:
        if c==')':
            res.append(')')
            return res
        else:
            # note how we use a recursive call to group the inner
            # matching parenthesis string and append it as a whole
            # to the list we are constructing.
            # Also note how we've already seen the leading '(' of this
            # inner group and consumed it from the iterator.
            res.append(groupMatching(it))
    return False

# function to turn a string of properly nested parentheses
# into a list of properly nested lists.
def group(s):
    if s[0]=='(':
        return groupMatching(iter(s[1:]))
    else: return False                  # If it starts with ')' it is not properly nested
```
Here I use an interator constructed from a string, but the `iter` 
function will equally well create an iterator from a list. 

Of course, *your code has to deal with curly braces instead of parentheses* and it must also *deal with
strings that contain tokens in addition to the curly braces.* And don't forget that strings representing
numbers and booleans should be converted to ints and bools at this stage as well. Again, I urge you to
not include the curly brace characters in the resulting code array. The structure of the code array itself
is sufficient for what we will do with it.

To illustrate the above point, consider this modified version of `groupMatching` and `group` which
doesn't put the paren characters into its result. Just the *structure* of the result is sufficient
to show the *structure* of the orginal string.

In [8]:
# The it argument is an iterator that returns left or right parenthesis characters. 
# The sequence of return characters should represent a string of properly nested 
# parentheses pairs, from which the leading '(' has already been removed. If the 
# parenteses are not properly nested, returns False.
def groupMatching(it):
    res = []
    for c in it:
        if c==')':
            return res
        else:
            # note how we use a recursive call to group the inner
            # matching parenthesis string and append it as a whole
            # to the list we are constructing.
            res.append(groupMatching(it))
    return False

# function to turn a string of properly nested parentheses
# into a list of properly nested lists.
def group(s):
    if s[0]=='(':
        return groupMatching(iter(s[1:]))
    else: return False                  # If it starts with ')' it is not properly nested
group('(()(()))')

[[], [[]]]

#### Your parsing implementation

In [9]:
# In this cell, write your parsing code; it takes a list of tokens produced by tokenize
# and returns a code array; copy this cell into your sps.ipynb notebook and write the 
# necessary code. Of course you may create additional functions to help you write parse()
#
def parse(tokens):
    pass

### interpret
We're now ready to write the `interpret` function. It takes a code array as argument, and changes the state of the operand and dictionary stacks according to what it finds there, doing any output indicated by the SPS program (using the `stack` and `=` operators) along the way. `interpret` may be called recursively from the `if` or `ifelse` operators, or when a name is looked up and its value is a code array.

In [10]:
# Copy this cell to your sps.ipynb and write the necessary code; again write
# auxiliary functions if you need them. This will probably be the largest
# function of the whole project, but it will have a very regular and obvious
# structure if you've followed the plan of the assignment.
def interpret(code): # code is a code array
    pass

### interpreter
Finally, we can write the `interpreter` function that treats a string as an SPS program and interprets it.

In [11]:
# Copy this cell to your sps.ipynb
def interpreter(s): # s is a string
    interpret(parse(tokenize(s)))


## Testing 
### First test the parsing
Before even attempting to run you full interpreter, make sure that your parsing is working
correctly. Do you get what you expect as the result of the following:
```
parse(tokenize(
'''
true {1} if
'''
))
```
Make sure that the result contains a python boolean and a python integer.
How about
```
parse(tokenize(
'''
true 
{-1}{1} 
ifelse
'''
))
```
Make sure that there are two nested code arrays.

You should know what the correct result is for the following more complicated example. Is
your code producing the right answer? There's not much point in going on until it is.
```
parse(tokenize(
"""
/fact{
   dictz begin
      /n exch def
         n 2 lt
         { 1}
         {n -1 add fact n mul }
      ifelse
   end
}def
5 fact =
"""
))
```
### Finally, test the full interpreter


In [12]:
# Copy this cell to your sps.ipynb. Add tests of your own, each in its own cell, and discuss how they provide
# evidence that your interpreter is working correctly. 
interpreter(
"""
/fact{
   dictz exch exch begin
      /n exch def
         n 2 lt
         { 1}
         {n 1 sub fact n mul }
      ifelse
   end
}def
5 fact =
"""
)